In [ ]:
from nbdev import *

In [ ]:
#default_exp data_model

Cells will be exported to drt.data_model,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
"""
Module declaring the data model of the SQLite databse. This uses the SQLAlchemy ORM with 
the hierachical join mapping paradigm. Basically the data group class contains all the 
common fields and is a table which is 1-1 joined with several sub table which contains 
data group type specific fields. The model also provides for a many-to-many mapping linking
tags to Data Groups.
"""
from sqlalchemy import Column, ForeignKey, Table, UniqueConstraint
from sqlalchemy import Integer, BigInteger, String, DateTime,  Unicode, UnicodeText, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relation, relationship, exc
import typing
from pathlib import Path

class BaseModel():
    """ Base model to allow for a more verbose representation of model objects. """
    def __repr__(self) -> str:
        return self._repr(id=self.id)

    def _repr(self, **fields: typing.Dict[str, typing.Any]) -> str:
        '''
        Helper for __repr__
        '''
        field_strings = []
        at_least_one_attached_attribute = False
        for key, field in fields.items():
            try:
                field_strings.append(f'{key}={field!r}')
            except exc.DetachedInstanceError:
                field_strings.append(f'{key}=DetachedInstanceError')
            else:
                at_least_one_attached_attribute = True
        if at_least_one_attached_attribute:
            return f"<{self.__class__.__name__}({','.join(field_strings)})>"
        return f"<{self.__class__.__name__} {id(self)}>"


Base = declarative_base(cls=BaseModel)

tag_link_table = Table('tag_link', Base.metadata,
    Column('tag_id', Integer, ForeignKey('Tag.id')),
    Column('data_group_id', Integer, ForeignKey('Data_Group.id'))
)

class Tag(Base):
    """
    Table containing tags to be used in the application to tag data groups. This will allow
    certain data groups to be tagged with values for easier retrieval and grouping when 
    organising the data groups.
    """
    __tablename__ = 'Tag'
    id = Column(Integer, primary_key=True)
    tag_name = Column(Unicode, nullable = False)
    data_groups = relationship("Data_Group",
                    secondary=tag_link_table, back_populates='tag_list')

    def __repr__(self):
        return self._repr(id=self.id,
                          tag_name=self.tag_name)

class Data_Group(Base):
    """
    The base data group table. This contains all of the common columns which exist for all 
    data groups. Each row is 1-1 bound with a row in any of the Delivery, Raw_Data, or Dataset
    tables. Those tables contain data group type specific code. This is using the SQLAlchemy
    ORM mapping pattern called Joined Table Inheritance mapping: 
    'https://docs.sqlalchemy.org/en/13/orm/inheritance.html#joined-table-inheritance' 
    """
    __tablename__ = 'Data_Group'
    id = Column(Integer, primary_key=True)
    type = Column(String, nullable = False)
    name = Column(Unicode, nullable=False)
    description = Column( UnicodeText)
    last_update = Column(DateTime, nullable=False)
    size = Column(BigInteger, nullable=False)
    num_files = Column(Integer, nullable=False)
    group_last_modified = Column(DateTime, nullable=False)
    group_hash = Column(String(260), nullable=False)
    tag_list = relationship("Tag",
                    secondary=tag_link_table, back_populates='data_groups')

    __table_args__ = (
        # this can be db.PrimaryKeyConstraint if you want it to be a primary key
        UniqueConstraint('name', 'type'),
      )

    # Code used to link the base table with the specific tables in the SQLAlchemy ORM
    __mapper_args__ = {
        'polymorphic_identity':'Data_Group',
        'polymorphic_on':type
    }


    def __repr__(self):
        return self._repr(id = self.id,
                            type = self.type,
                            name = self.name)

class Delivery(Data_Group):
    """
    Table containing Delivery data group type specific columns. Links to the
    Data_Group table as it's parent table.
    Objects instantiated with this class will contain all the columns.
    """
    __tablename__ = 'Delivery'
    id = Column(Integer, ForeignKey('Data_Group.id'), primary_key=True)
    date_received = Column(Date)
    delivery_source = Column(UnicodeText)

    # SQLAlchemy ORM code to link this table to it's parent
    __mapper_args__ = {
        'polymorphic_identity':'Delivery',
    }

    def __repr__(self):
        return self._repr(id = self.id,
                          type = self.type,
                          name = self.name)

    def document(self):
        template_text = (Path(__file__).resolve().parent / 'templates' / 'delivery.rst').read_text()
        template_text = template_text.replace("========\n{{Name}}\n========", ("="*len(self.name)) + "\n" + self.name + "\n" + ("="*len(self.name)) )
        template_text = template_text.replace("{{Date Received}}", str(self.date_received))
        template_text = template_text.replace("{{Source of Delivery}}", str(self.delivery_source))
        template_text = template_text.replace("{{Description data goes here}}", str(self.description))
        links = ''
        for group in self.extracted_data:
            links = links + '- ' + group.name + '\n'
        template_text = template_text.replace("{{data_groups}}", links)
        template_text = template_text.replace("{{Tags}}", str(self.tag_list))
        template_text = template_text.replace("{{System Files Last Modified}}", str(self.group_last_modified))
        template_text = template_text.replace("{{System Size}}", str(self.size))
        template_text = template_text.replace("{{System Number of Files}}", str(self.num_files))
        template_text = template_text.replace("{{System Group Hash}}", str(self.group_hash))
        template_text = template_text.replace("{{System Last Update}} ", str(self.last_update))
        return template_text


class Raw_Data(Data_Group):
    """
    Table with raw data group type specific columns. Links to the
    Data_Group table as it's parent table.
    Objects instantiated with this class will contain all the columns.
    """
    __tablename__ = 'Raw_Data'
    id = Column(Integer, ForeignKey('Data_Group.id'), primary_key=True)
    statistics_report = Column( UnicodeText)
    source_delivery_id = Column(Integer, ForeignKey('Delivery.id'))
    source = relationship("Delivery", backref='extracted_data', foreign_keys=[source_delivery_id])

    # SQLAlchemy ORM code to link this table to it's parent
    __mapper_args__ = {
        'polymorphic_identity':'Raw_Data',
    }

    def __repr__(self):
        return self._repr(id = self.id,
                          type = self.type,
                          name = self.name,
                          source = self.source)

    def document(self):
        template_text = (Path(__file__).resolve().parent / 'templates' / 'raw_data.rst').read_text()
        template_text = template_text.replace("========\n{{Name}}\n========", ("="*len(self.name)) + "\n" + self.name + "\n" + ("="*len(self.name)) )
        template_text = template_text.replace("{{statistics_report}}", str(self.statistics_report))
        template_text = template_text.replace("{{analyst populated description}}", str(self.description))
        # links = ''
        # for group in self.links:
        #     links = links + '- ' + group.name + '\n'
        # template_text = template_text.replace("{{links}}", links)
        template_text = template_text.replace("{{Tags}}", str(self.tag_list))
        template_text = template_text.replace("{{group_last_modified}}", str(self.group_last_modified))
        template_text = template_text.replace("{{size}}", str(self.size))
        template_text = template_text.replace("{{numfiles}}", str(self.num_files))
        template_text = template_text.replace("{{group_hash}}", str(self.group_hash))
        template_text = template_text.replace("{{last_update}} ", str(self.last_update))
        return template_text

class Dataset(Data_Group):
    """
    Table with dataset group type specific columns. Links to the
    Data_Group table as it's parent table. 
    Objects instantiated with this class will contain all the columns.
    """
    __tablename__ = 'Dataset'
    id = Column(Integer, ForeignKey('Data_Group.id'), primary_key=True)
    dataset_report = Column( UnicodeText)

    # SQLAlchemy ORM code to link this table to it's parent
    __mapper_args__ = {
        'polymorphic_identity':'Dataset',
    }

    def __repr__(self):
        return self._repr(id = self.id,
                          type = self.type,
                          name = self.name)

    def document(self):
        template_text = (Path(__file__).resolve().parent / 'templates' / 'raw_data.rst').read_text()
        template_text = template_text.replace("========\n{{Name}}\n========", ("="*len(self.name)) + "\n" + self.name + "\n" + ("="*len(self.name)) )
        template_text = template_text.replace("{{statistics_report}}", str(self.dataset_report))
        template_text = template_text.replace("{{analyst populated description}}", str(self.description))
        # links = ''
        # for group in self.extracted_data:
        #     links = links + '- ' + group.name + '\n'
        # template_text = template_text.replace("{{links}}", links)
        template_text = template_text.replace("{{Tags}}", str(self.tag_list))
        template_text = template_text.replace("{{group_last_modified}}", str(self.group_last_modified))
        template_text = template_text.replace("{{size}}", str(self.size))
        template_text = template_text.replace("{{numfiles}}", str(self.num_files))
        template_text = template_text.replace("{{group_hash}}", str(self.group_hash))
        template_text = template_text.replace("{{last_update}} ", str(self.last_update))
        return template_text